In [3]:
if "descended" not in locals():
    descended = 1
    %cd ".."
    
import utils

import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [35]:
os.listdir("data/lilith/dv")

['.DS_Store',
 'dvfiles_curl_filtered.sh',
 'lilith4_sector-1-sector-4_dvfiles_curl.sh',
 'tess2018192011050-s0001-s0004-0000000038680132-00062_dvt.fits.gz']

In [36]:
load_path = "data/lilith/dv"
fname = "tess2018192011050-s0001-s0004-0000000038680132-00062_dvt.fits.gz"

hdu = fits.open(load_path + "/" + fname)

In [43]:
hdu[1].header

XTENSION= 'BINTABLE'           / marks the beginning of a new HDU               
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   44 / length of first array dimension                
NAXIS2  =                81195 / length of second array dimension               
PCOUNT  =                    0 / group parameter count (not used)               
GCOUNT  =                    1 / group count (not used)                         
TFIELDS =                   10 / number of table fields                         
TTYPE1  = 'TIME    '           / column title: data time stamps                 
TFORM1  = 'D       '           / column format: 64-bit floating point           
TUNIT1  = 'BJD - 2457000, days' / column units: Barycenter corrected TESS Julian
TDISP1  = 'D14.7   '           / column display format                          
TTYPE2  = 'TIMECORR'        

In [44]:
hdu[0].header["SECTORS"],  hdu[0].header["TSTOP"]- hdu[0].header["TSTART"]

('01111', 112.77178240443004)

In [46]:
hdu[1].header["TPERIOD"], hdu[1].header["TEPOCH"], hdu[1].header["TDUR"]/24, hdu[1].header["TDEPTH"]*1e-6

(51.45489169327453,
 1336.4581651207177,
 0.4780053649703076,
 0.006175933691927643)

In [47]:
hdu[1].header["TSNR"], hdu[1].header["NTRANS"]

(9.799813270568848, 2)

In [48]:
!ls

README.md        detection        models           training
__pycache__      experiments      requirements.txt utils.py
data             figures          results          venv
dataloading      lilith           simulations      visualize.py
